In [1]:
from googleapiclient.discovery import build
import pandas as pd
from IPython.display import JSON

In [2]:
api_key = input("Please enter your YouTube Data API key: ")
channel_ids=['UCX6OQ3DkcsbYNE6H8uQQuVA',
             'UCChmJrVa8kDg05JfCmxpLRw',
             'UCkX0NXCP9kCLtk30kjTXIWg',
             'UCXqJrBG563pZnaqLoxHjUDg',
             'UCHoLIMtg_OigNlJmjHX9J8Q',
             'UCMOLiZpKXp-w5CNfqJObQUw',
             'UCtAXJ4DNpshfVjLIMq9pIRw']

youtube= build('youtube','v3',developerKey=api_key)

In [3]:
def get_channel(youtube,channel_ids):
    
    all_data = []
    request = youtube.channels().list(
                part='snippet,contentDetails,statistics',
                id=','.join(channel_ids)
                )
    response = request.execute() 
    

    for i in range(len(response['items'])):
        data = dict(channelName = response['items'][i]['snippet']['title'],
                    subscribers = response['items'][i]['statistics']['subscriberCount'],
                    views = response['items'][i]['statistics']['viewCount'],
                    totalVideos = response['items'][i]['statistics']['videoCount'],
                    playlistId = response['items'][i]['contentDetails']['relatedPlaylists']['uploads'])
        all_data.append(data)
    
    return pd.DataFrame(all_data)




In [4]:
channel_stats=get_channel(youtube,channel_ids)
channel_stats

,channelName,subscribers,views,totalVideos,playlistId
0,Something_beyond,18,8533,46,UUkX0NXCP9kCLtk30kjTXIWg
1,coder2j,9780,906539,48,UUHoLIMtg_OigNlJmjHX9J8Q
2,Darshil Parmar,127000,5860986,149,UUChmJrVa8kDg05JfCmxpLRw
3,MrBeast,255000000,47943742180,790,UUX6OQ3DkcsbYNE6H8uQQuVA
4,KALEO,1940000,1538340396,68,UUXqJrBG563pZnaqLoxHjUDg
5,KEEMOKAZI,9670000,10508351483,1317,UUMOLiZpKXp-w5CNfqJObQUw
6,Rohit & Kanupriya,2230000,2120854417,321,UUtAXJ4DNpshfVjLIMq9pIRw


In [5]:
playlistid=channel_stats['playlistId']
playlistid

0    UUkX0NXCP9kCLtk30kjTXIWg
1    UUHoLIMtg_OigNlJmjHX9J8Q
2    UUChmJrVa8kDg05JfCmxpLRw
3    UUX6OQ3DkcsbYNE6H8uQQuVA
4    UUXqJrBG563pZnaqLoxHjUDg
5    UUMOLiZpKXp-w5CNfqJObQUw
6    UUtAXJ4DNpshfVjLIMq9pIRw
Name: playlistId, dtype: object

In [6]:
def get_video_ids(youtube, playlist_id,video_ids):

    request = youtube.playlistItems().list(
                part='contentDetails',
                playlistId = playlist_id,
                maxResults = 50)
    response = request.execute()
    
    
    for i in range(len(response['items'])):
        video_ids.append(response['items'][i]['contentDetails']['videoId'])
        
    next_page_token = response.get('nextPageToken')
    more_pages = True
    
    while more_pages:
        if next_page_token is None:
            more_pages = False
        else:
            request = youtube.playlistItems().list(
                        part='contentDetails',
                        playlistId = playlist_id,
                        maxResults = 50,
                        pageToken = next_page_token)
            response = request.execute()
    
            for i in range(len(response['items'])):
                video_ids.append(response['items'][i]['contentDetails']['videoId'])
            
            next_page_token = response.get('nextPageToken')
        
    return video_ids


In [7]:
video_ids = []
for i in range(len(playlistid)):
    playlist_id=playlistid[i]
    Video_ids=get_video_ids(youtube, playlist_id,video_ids)
    

len(Video_ids)

2740

In [8]:
def get_video_details(youtube, video_ids):    
    all_video_info = []
    
    for i in range(0, len(video_ids), 50):
        request = youtube.videos().list(
            part="snippet,contentDetails,statistics",
            id=','.join(video_ids[i:i+50])
        )
        response = request.execute() 

        for video in response['items']:
            stats_to_keep = {'snippet': ['channelTitle', 'title', 'description', 'tags', 'publishedAt'],
                             'statistics': ['viewCount', 'likeCount', 'favouriteCount', 'commentCount'],
                             'contentDetails': ['duration', 'definition', 'caption']
                            }
            video_info = {}
            video_info['video_id'] = video['id']

            for k in stats_to_keep.keys():
                for v in stats_to_keep[k]:
                    try:
                        video_info[v] = video[k][v]
                    except:
                        video_info[v] = None

            all_video_info.append(video_info)
            
    return pd.DataFrame(all_video_info)

In [9]:
videoDetails=get_video_details(youtube, Video_ids)
videoDetails

,video_id,channelTitle,title,description,tags,publishedAt,viewCount,likeCount,favouriteCount,commentCount,duration,definition,caption
0,YZSv1Q8zuWo,Something_beyond,Elon Musk's Space X With Kaleo-Way-Down-We-Go,Join us as we witness the incredible journey o...,"[spacex, space x, spacex night launch, spacex ...",2024-04-27T07:45:27Z,8,3,None,0,PT3M38S,hd,false
1,njzjxMftEe8,Something_beyond,Catching super heavy,,"[SpaceExplorer, space, spaceX, falcon9, falcon...",2024-04-07T15:51:26Z,2,1,None,0,PT1M,hd,false
2,hmxfn4cTdlQ,Something_beyond,Watch Falcon 9 launch the Eutelsat Group EUTEL...,Witness the spectacular Falcon 9 launch as it ...,"[SpaceExplorer, space, spaceX, falcon9, falcon...",2024-03-31T15:02:09Z,3,2,None,0,PT1M,hd,false
3,iXVGrwRxV2c,Something_beyond,Full duration static fire of all six Raptor en...,Watch this incredible footage of the full dura...,"[SpaceExplorer, space, spaceX, falcon9, falcon...",2024-03-27T12:36:50Z,5,2,None,0,PT15S,hd,false
4,vW_LAxHJY_w,Something_beyond,Falcon 9 launches Starlink satellites to orbit...,Watch the latest Falcon 9 launch as it sends a...,"[SpaceExplorer, space, spaceX, falcon9, falcon...",2024-03-27T12:32:30Z,438,11,None,0,PT47S,hd,false
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2735,TRM_e1O7TVU,Rohit & Kanupriya,The problem with ordering food online,,"[food delivery, ordering food, ordering food o...",2021-05-30T16:49:42Z,6561752,211070,None,1195,PT21S,hd,false
2736,U_xZFqXOyFY,Rohit & Kanupriya,This break up didn't hurt,Another one from the 'me annoying me' series.,"[Funny, tiktok, funny videos, Instagram reels,...",2021-05-26T15:37:56Z,496601,15450,None,58,PT13S,hd,false
2737,k7dmrSbCpy4,Rohit & Kanupriya,My daily routine a.k.a The Vicious Cycle,My daily routine. How I wish I could somehow t...,"[anxiety, motivation, netflix, daily routine, ...",2021-05-19T20:29:25Z,201913,10497,None,87,PT31S,hd,false
2738,cffqyq-U2Og,Rohit & Kanupriya,Every group has THIS guy.,Every group has this annoying person. The one ...,"[Questions, friend, what would you do, annoyin...",2021-05-19T20:26:16Z,206882,3084,None,42,PT1M20S,hd,false


In [10]:

def get_comments_in_videos(youtube, video_ids):
    all_comments = []
    
    for video_id in video_ids:
        try:   
            request = youtube.commentThreads().list(
                part="snippet,replies",
                videoId=video_id
            )
            response = request.execute()
        
            comments_in_video = [comment['snippet']['topLevelComment']['snippet']['textOriginal'] for comment in response['items'][0:10]]
            comments_in_video_info = {'video_id': video_id, 'comments': comments_in_video}

            all_comments.append(comments_in_video_info)
            
        except: 
            # When error occurs - most likely because comments are disabled on a video
            print('Could not get comments for video ' + video_id)
        
    return pd.DataFrame(all_comments)  

comments=get_comments_in_videos(youtube, Video_ids)
comments

Could not get comments for video MO-a-5G_ZAI
Could not get comments for video AS5CxLCWq-Q
Could not get comments for video wqdvsLccHN8
Could not get comments for video JOxQolirlcc
Could not get comments for video a_Kgsb6QIz4
Could not get comments for video yQo0noQwsKQ


,video_id,comments
0,YZSv1Q8zuWo,[]
1,njzjxMftEe8,[]
2,hmxfn4cTdlQ,[]
3,iXVGrwRxV2c,[]
4,vW_LAxHJY_w,[]
...,...,...
2729,TRM_e1O7TVU,"[Modi hai toh mumkin hai❤️, No will power char..."
2730,U_xZFqXOyFY,"[That creepy smile, I ended 4 years relationsh..."
2731,k7dmrSbCpy4,"[Finished all of ur shorts😂😂, 😊, Who came to w..."
2732,cffqyq-U2Og,"[I'm that guy but I don't have any group, When..."


In [11]:
videoDetails.to_csv('Data\\apiData.csv', index=False)
comments.to_csv('Data\\commentsData.csv', index=False)